# Introduction 

In this data analysis I will try to figure our who is the most valuable player based on their influence into winning, stats in comparison to their salary. 

Based on that we will be able to figure out which player is Overpaid and which is Underpaid.


In [1]:
#First we need to import Beautiful Soup library that will allow us to do web scraping as well as pandas that will allow us to
#work with our dataframe later on.
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

# Data scraping

In [2]:
r = requests.get('https://www.basketball-reference.com/leagues/NBA_2020_advanced.html')
soup = bs(r.content)

print(str(soup)[0:500])

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202010091" rel="dns-prefetch"/>
<title>2019-20 NBA Player Stats: Advanced | Basketball-Reference.com</title>
<meta conte


As we can see this is not the most beautiful way to show this data. We have to change it!

In [3]:
print(str(soup.prettify())[0:500])

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
  <link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202010091" rel="dns-prefetch"/>
  <title>
   2019-20 NBA Player Stats: Advanced | Basketball-Reference.com
  </


Still not perfect but we can at least distinguish some parts of our html code.

Now it is time to change our HTML code to dataframe using Pandas and BeautifulSoup

In [4]:
#First we need to find our headers for the dataframe
soup.findAll('tr', limit=2)
headers = [th.getText() 
           for th in soup.findAll('tr', limit=2)[0].findAll('th')]

headers = headers[1:]
headers

['Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'MP',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 '\xa0',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 '\xa0',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']

In [5]:
#Great! Now we need to scrape data of each player row by row using loop
rows = soup.findAll('tr')[1:]
players = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

In [6]:
#Now it is time to input our data into each row using Pandas
stats = pd.DataFrame(players, columns = headers)
stats.head(10)

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP
0,Steven Adams,C,26,OKC,63,1680,20.5,.604,.006,.421,...,,3.8,2.7,6.5,.185,,1.9,1.1,2.9,2.1
1,Bam Adebayo,PF,22,MIA,72,2417,20.3,.598,.018,.484,...,,4.6,3.9,8.5,.168,,1.4,2.0,3.4,3.3
2,LaMarcus Aldridge,C,34,SAS,53,1754,19.7,.571,.198,.241,...,,3.0,1.4,4.5,.122,,1.8,-0.5,1.4,1.5
3,Kyle Alexander,PF,23,MIA,2,13,4.7,.500,.000,.000,...,,0.0,0.0,0.0,-0.003,,-6.1,-3.5,-9.6,0.0
4,Nickeil Alexander-Walker,SG,21,NOP,47,591,8.9,.473,.500,.139,...,,-0.7,0.4,-0.2,-0.020,,-3.2,-1.4,-4.6,-0.4
5,Grayson Allen,SG,24,MEM,38,718,12.0,.609,.562,.179,...,,0.8,0.4,1.2,.082,,-0.1,-1.2,-1.3,0.1
6,Jarrett Allen,C,21,BRK,70,1852,20.7,.664,.013,.581,...,,5.2,2.9,8.2,.212,,1.4,1.0,2.3,2.0
7,Kadeem Allen,SG,27,NYK,10,117,14.0,.512,.364,.250,...,,0.1,0.1,0.1,.053,,-0.1,-0.6,-0.7,0.0
8,Al-Farouq Aminu,PF,29,ORL,18,380,7.6,.395,.419,.337,...,,-0.4,0.5,0.1,.014,,-4.4,1.4,-3.0,-0.1
9,Justin Anderson,SF,26,BRK,10,107,3.2,.352,.763,.105,...,,-0.2,0.1,-0.1,-0.054,,-6.8,-0.9,-7.7,-0.2


Amazing we have our dataframe ready but to our later purposes we don't need every column. 
Now it is time to develop our evaluation model which will be used to evaluate how much a player is "worth".

# Modeling

For this project I came up with a formula that in my opinion fits best to the value of each player. It will be weighted average
of a few factors that contribute to player performance and team wins.

My formula is:
    
[(G/Gt)*9+[(50-Age)/50]*4+(Mp/Mt)*9+(PER/15)*9+(TS%/100)*9+[(TRB%+AST%+STL%+BLK%)/100]*5-(TOV%/100)*7+(USG%/100)*7(WS48)*10]/80

The data included here and it's meaning to players value:

G- number of games played. It is very simple if the player is not playing he doesn't add value to a win.

Gt - number of games played in a season which for 2020 is 71

Age - age of a player. If your name is not Lebron James after certain age teams must concider your age when playing back-to-back etc. This contributes to the game plan and season plan

Mp-minutes played. As with games played you add value to the team when you are out on the floor.

Mt - total number of minutes in a season on average (without OT) which is 71*48= 3408

PER - a measeure of Per minute production by a player where average is 15

TS% - true shooting percentage that takes into account every shot that a player took

TRB% - total rebound percentage 

AST% - total assist percentage 

STL% - total steals percentage

BLK% - total blocks percentage

TOV% - total turnovers percentage - I gave this a weight of 7 because every NBA fan knows how deadly lost posessions are 
and even with other stats being perfect it is almost impossible to win with great amount of turnovers 

USG% - usage percentage. An estimate of plays that used specific player in them.

WS/48 - estimate of numbers of wins contributed by a player by 48 minutes.

# Adapting dataframe to our needs

In [7]:
#First we need to drop columns that we will not be using later on.
stats=stats.drop(['Tm', '3PAr','FTr','ORB%','DRB%','\xa0', 'OWS','DWS','WS','\xa0','OBPM','DBPM','BPM','VORP','Pos'],axis=1)
stats.head()


,Player,Age,G,MP,PER,TS%,TRB%,AST%,STL%,BLK%,TOV%,USG%,WS/48
0,Steven Adams,26,63,1680,20.5,.604,19.2,13.2,1.5,3.4,14.2,17.3,.185
1,Bam Adebayo,22,72,2417,20.3,.598,17.0,24.2,1.7,3.8,17.6,21.2,.168
2,LaMarcus Aldridge,34,53,1754,19.7,.571,12.0,11.4,1.0,4.4,7.8,23.4,.122
3,Kyle Alexander,23,2,13,4.7,.500,12.9,0.0,0.0,0.0,33.3,10.2,-0.003
4,Nickeil Alexander-Walker,21,47,591,8.9,.473,7.5,21.1,1.3,1.1,16.1,23.3,-0.020


In [8]:
for column in stats.columns:
    if column == 'Player':
        continue
    else:
        stats[column]=pd.to_numeric(stats[column], errors='coerce')
        
stats.head()

,Player,Age,G,MP,PER,TS%,TRB%,AST%,STL%,BLK%,TOV%,USG%,WS/48
0,Steven Adams,26.0,63.0,1680.0,20.5,0.604,19.2,13.2,1.5,3.4,14.2,17.3,0.185
1,Bam Adebayo,22.0,72.0,2417.0,20.3,0.598,17.0,24.2,1.7,3.8,17.6,21.2,0.168
2,LaMarcus Aldridge,34.0,53.0,1754.0,19.7,0.571,12.0,11.4,1.0,4.4,7.8,23.4,0.122
3,Kyle Alexander,23.0,2.0,13.0,4.7,0.500,12.9,0.0,0.0,0.0,33.3,10.2,-0.003
4,Nickeil Alexander-Walker,21.0,47.0,591.0,8.9,0.473,7.5,21.1,1.3,1.1,16.1,23.3,-0.020


In [9]:
#Great now it is time to calculate our player value by previously developed model. Let's call it PV (Player Value)

game=(stats['G']/71)*9
age=(50-(stats['Age']/50))*2
minutes = (stats['MP']/3408)*9
per = stats['PER']*(8/15)
ts= stats['TS%']*(9/100)
ave = (stats['TRB%']+stats['AST%']+stats['STL%']+stats['BLK%'])*(5/100)
tov=stats['TOV%']*(6/100)
usg = stats['USG%']*(7/100)
ws = stats['WS/48']*10

value = (game + age+minutes +per +ts + ave -tov +usg+ws)/80
stats['PV']=value

In [10]:
stats=stats.sort_values('PV', ascending = False)
stats.head(50)

,Player,Age,G,MP,PER,TS%,TRB%,AST%,STL%,BLK%,TOV%,USG%,WS/48,PV
12,Giannis Antetokounmpo,25.0,63.0,1917.0,31.9,0.613,22.1,34.2,1.5,3.0,13.2,37.5,0.279,1.709749
266,James Harden,30.0,68.0,2483.0,29.1,0.626,9.4,35.9,2.3,2.1,14.2,36.3,0.254,1.703341
183,Luka Dončić,20.0,61.0,2047.0,27.6,0.585,14.7,45.7,1.5,0.6,14.8,36.8,0.207,1.674923
390,Damian Lillard,29.0,66.0,2474.0,26.9,0.627,6.0,34.4,1.3,0.8,11.0,30.3,0.225,1.674734
347,Nikola Jokić,24.0,73.0,2336.0,24.9,0.605,17.1,35.2,1.8,1.7,15.8,26.6,0.202,1.669012
331,LeBron James,35.0,67.0,2316.0,25.5,0.577,12.4,49.1,1.6,1.4,15.1,31.5,0.204,1.667814
168,Anthony Davis,26.0,62.0,2131.0,27.4,0.610,14.8,15.1,2.0,6.0,10.4,29.3,0.250,1.661713
387,Kawhi Leonard,28.0,57.0,1848.0,26.9,0.589,11.3,26.4,2.6,1.6,10.2,33.0,0.226,1.642979
672,Trae Young,21.0,60.0,2120.0,23.9,0.595,6.5,45.6,1.4,0.3,16.2,34.9,0.133,1.633193
651,Hassan Whiteside,30.0,67.0,2008.0,25.0,0.644,23.6,5.9,0.6,8.4,12.9,19.6,0.204,1.631876


Now it is time to search for data about current players salaries.


In [11]:
r2 = requests.get('https://www.basketball-reference.com/contracts/players.html')
soup2 = bs(r2.content)
#First we need to find our headers for the dataframe
soup2.findAll('tr', limit=2)
headers2 = [th.getText() 
           for th in soup2.findAll('tr', limit=2)[0].findAll('th')]

headers2 = headers2[1:]
headers2



['Salary', '']